In [2]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import glm
import collections
from statsmodels.sandbox.stats.multicomp import multipletests


In [3]:
df=pd.read_csv('Final_poisson.csv').iloc[:,1:]

In [27]:
df_drop=df.drop_duplicates(subset='LAT_LON')

In [41]:
df_drop.columns


Index(['FECHA_ACCIDENTE', 'AÑO_ACCIDENTE', 'MES_ACCIDENTE', 'DIA_ACCIDENTE',
       'HORA_ACCIDENTE', 'GRAVEDAD_ACCIDENTE', 'CLASE_ACCIDENTE',
       'SITIO_EXACTO_ACCIDENTE', 'CANT_HERIDOS_EN _SITIO_ACCIDENTE',
       'CANT_MUERTOS_EN _SITIO_ACCIDENTE', 'LONGITUD', 'LATITUD', 'POT',
       'PIEZA_URBANA', 'BUS_SUBIDA', 'BUS_BAJADA', 'TRAFICO', 'ES_FESTIVO',
       'TIPO_FESTIVO', 'DIA_DEL_AÑO', 'SEMANA_DEL_AÑO', 'CUATRIMESTRE_DEL_AÑO',
       'JUNIOR_LOCAL', 'JUNIOR_VISITANTE', 'RESULTADO_LOCAL',
       'RESULTADO_VISITANTE', 'JUNIOR_GANO', 'RELEVANCIA_PARTIDO',
       'HIPOTESIS_ACCIDENTE', 'VEHICULO_MOTOCICLETA', 'VEHICULO_AUTOMOVIL',
       'VEHICULO_CAMIONETA', 'VEHICULO_BUS', 'VEHICULO_CAMION',
       'VEHICULO_CAMPERO', 'VEHICULO_BUSETA', 'VEHICULO_BICICLETA',
       'VEHICULO_MICROBUS', 'VEHICULO_TRACTO_CAMION', 'VEHICULO_VOLQUETA',
       'VEHICULO_MOTOCARRO', 'VEHICULO_DESCONOCIDO', 'VEHICULO_CUATRIMOTO',
       'VEHICULO_MAQUINARIA_AGRICOLA', 'VEHICULO_TRACCION_ANIMAL',
    

In [4]:
import time

In [73]:
def conteo_perf(newdf=df):
    conteos=newdf[['LAT_LON','SITIO_EXACTO_ACCIDENTE']].groupby(['LAT_LON','SITIO_EXACTO_ACCIDENTE']).size().reset_index(name='count')
    grp_conteos = conteos.groupby('LAT_LON')['count'].agg(['sum','max'])
    grp_conteos['sum_rate'] = grp_conteos['max'] / grp_conteos['sum']
    filtered_conteos = grp_conteos[grp_conteos['sum_rate'] > 0.3]
    return_df = filtered_conteos.drop(['max', 'sum_rate'], axis=1)
    return_df.rename(columns = {'sum':'NACCIDENTS'}, inplace = True) 
    fusion=pd.merge(return_df,df_drop,on='LAT_LON',how='inner')[['NACCIDENTS','AÑO_ACCIDENTE','MES_ACCIDENTE',
                                                     'CLASE_ACCIDENTE','LONGITUD', 'LATITUD',
                                                     'POT', 'PIEZA_URBANA', 'BUS_SUBIDA','BUS_BAJADA', 
                                                     'TRAFICO','ES_FESTIVO','SOLO_HORA']].reset_index().iloc[:,1:]
    return(fusion)

start_time = time.time()
b=conteo_perf()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.06024885177612305 seconds ---


In [74]:
b

,NACCIDENTS,AÑO_ACCIDENTE,MES_ACCIDENTE,CLASE_ACCIDENTE,LONGITUD,LATITUD,POT,PIEZA_URBANA,BUS_SUBIDA,BUS_BAJADA,TRAFICO,ES_FESTIVO,SOLO_HORA
0,1,2019,4,Choque,-74.810748,10.915367,protegida,ninguna,0,0,0.003922,YES,11
1,1,2018,5,Choque,-74.795213,10.917122,protegida,ninguna,3,0,0.003922,NO,14
2,1,2018,1,Choque,-74.807522,10.917998,protegida,ninguna,2,0,0.705882,NO,11
3,1,2019,6,Choque,-74.797185,10.918646,protegida,ninguna,2,0,0.003922,NO,15
4,1,2019,5,Choque,-74.826326,10.921112,protegida,ninguna,0,0,0.294118,NO,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5825,1,2016,9,Choque,-74.827026,11.040540,residencial,ribera occidental,1,0,0.705882,NO,11
5826,1,2018,4,Atropello,-74.824731,11.040967,residencial,ribera occidental,1,0,0.003922,NO,20
5827,1,2017,5,Atropello,-74.827933,11.041935,residencial,ribera occidental,1,0,0.705882,NO,15
5828,1,2018,1,Choque,-74.826731,11.042032,residencial,ribera occidental,1,0,0.701961,NO,9
